# Config

In [ ]:
# Required
ENV = "MiniGrid-Fetch-8x8-N3-v0"
MODEL = "EXPERIMENTAL"

# Optional
SEED = 0
SHIFT = 0
ARGMAX = False
PAUSE = 0.1
GIF = None
EPS = 1000000
MEMORY = False
TEXT = False

# Visualize

In [ ]:
import argparse
import numpy

import utils
from utils import device

# Set seed for all randomness sources

utils.seed(SEED)

# Set device

print(f"Device: {device}\n")

# Load environment

env = utils.make_env(ENV, SEED, render_mode="human")
for _ in range(SHIFT):
    env.reset()
print("Environment loaded\n")

# Load agent

model_dir = utils.get_model_dir(MODEL)
agent = utils.Agent(env.observation_space, env.action_space, model_dir,
                    argmax=ARGMAX, use_memory=MEMORY, use_text=TEXT)
print("Agent loaded\n")

# Run the agent

if GIF:
    from array2gif import write_gif

    frames = []

# Create a window to view the environment
env.render()

for episode in range(EPS):
    obs, _ = env.reset()

    while True:
        env.render()
        if GIF:
            frames.append(numpy.moveaxis(env.get_frame(), 2, 0))

        action = agent.get_action(obs)
        obs, reward, terminated, truncated, _ = env.step(action)
        done = terminated | truncated
        agent.analyze_feedback(reward, done)

        if done:
            break

if GIF:
    print("Saving gif... ", end="")
    write_gif(numpy.array(frames), GIF+".gif", fps=1/PAUSE)
    print("Done.")
